In [2]:
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel

In [4]:
# List of medical keywords (expand this based on your feature and domain knowledge)
medical_keywords = ['medicine', 'Medicine', 'medicines', 'medication', 'psychiatry', 'medicinal', 'Medic', 'medications', 'medical', 'physician', 'doctor', 'patient', 'hospital', 'diagnosis', 'treatment']

def keyword_score(text, keywords):
    return sum(keyword.lower() in text.lower() for keyword in keywords)

def get_embedding(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

In [5]:
# Load pre-trained model for embeddings
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

In [9]:
# Texts to compare
baseline_text = "I think the most challenging career path is lawyer."
steered_text = "I think working in medicine is the most challenging career path."
reference_text = " ".join(medical_keywords)

# Keyword matching
baseline_score = keyword_score(baseline_text, medical_keywords)
steered_score = keyword_score(steered_text, medical_keywords)

# Semantic similarity
baseline_embedding = get_embedding(baseline_text, model, tokenizer)
steered_embedding = get_embedding(steered_text, model, tokenizer)
reference_embedding = get_embedding(reference_text, model, tokenizer)

baseline_similarity = cosine_similarity([baseline_embedding], [reference_embedding])[0][0]
steered_similarity = cosine_similarity([steered_embedding], [reference_embedding])[0][0]

print(f"Comparing baseline text '{baseline_text}' to steered text '{steered_text}'")
print(f"Keyword Scores - Baseline: {baseline_score}, Steered: {steered_score}")
print(f"Semantic Similarities - Baseline: {baseline_similarity:.4f}, Steered: {steered_similarity:.4f}")

if steered_score > baseline_score and steered_similarity > baseline_similarity:
    print("The steered output is more aligned with the medical feature.")
elif steered_score == baseline_score and steered_similarity == baseline_similarity:
    print("There is no significant difference in alignment between the outputs.")
else:
    print("The baseline output is more aligned with the medical feature.")

Comparing baseline text 'I think the most challenging career path is lawyer.' to steered text 'I think working in medicine is the most challenging career path.'
Keyword Scores - Baseline: 0, Steered: 3
Semantic Similarities - Baseline: 0.3848, Steered: 0.4514
The steered output is more aligned with the medical feature.


In [11]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
import torch

# List of medical keywords (expand this based on your feature and domain knowledge)
medical_keywords = ['medicine', 'Medicine', 'medicines', 'medication', 'psychiatry', 'medicinal', 'Medic', 'medications', 'medical', 'physician', 'doctor', 'patient', 'hospital', 'diagnosis', 'treatment', 'nurse', 'surgery', 'clinic', 'therapy', 'health']

def keyword_score(text, keywords):
    return sum(keyword.lower() in text.lower() for keyword in keywords)

def get_embedding(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Load pre-trained model for embeddings
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

# Reference text for similarity comparison
reference_text = " ".join(medical_keywords)
reference_embedding = get_embedding(reference_text, model, tokenizer)

def compare_texts(baseline_text, steered_text):
    # Keyword matching
    baseline_score = keyword_score(baseline_text, medical_keywords)
    steered_score = keyword_score(steered_text, medical_keywords)

    # Semantic similarity
    baseline_embedding = get_embedding(baseline_text, model, tokenizer)
    steered_embedding = get_embedding(steered_text, model, tokenizer)

    baseline_similarity = cosine_similarity([baseline_embedding], [reference_embedding])[0][0]
    steered_similarity = cosine_similarity([steered_embedding], [reference_embedding])[0][0]

    return {
        'baseline_score': baseline_score,
        'steered_score': steered_score,
        'baseline_similarity': baseline_similarity,
        'steered_similarity': steered_similarity
    }

def evaluate_alignment(results):
    keyword_diff = results['steered_score'] - results['baseline_score']
    similarity_diff = results['steered_similarity'] - results['baseline_similarity']
    
    if keyword_diff > 0 or similarity_diff > 0.01:  # 0.01 is a small threshold to account for minor fluctuations
        return "The steered output is more aligned with the medical feature."
    elif keyword_diff == 0 and abs(similarity_diff) <= 0.01:
        return "There is no significant difference in alignment between the outputs."
    else:
        return "The baseline output is more aligned with the medical feature."

# Test cases
test_cases = [
    ("I think the most challenging career path is lawyer.", "I think the most challenging career path is doctor."),
    ("I think the most challenging career path is lawyer.", "I think the most challenging career path is nurse."),
    ("The project requires careful planning and execution.", "The surgery requires careful planning and execution."),
    ("She works long hours at the office.", "She works long hours at the hospital."),
    ("He's studying to become a teacher.", "He's studying to become a pharmacist."),
    ("The conference is about new technological advancements.", "The conference is about new medical advancements."),
    ("She's an expert in resolving conflicts.", "She's an expert in diagnosing rare diseases."),
    ("The book discusses modern management techniques.", "The book discusses modern treatment techniques."),
    ("They're developing a new app for smartphones.", "They're developing a new drug for cancer treatment."),
    ("He's passionate about environmental conservation.", "He's passionate about mental health awareness.")
]

for i, (baseline, steered) in enumerate(test_cases, 1):
    print(f"\nTest Case {i}:")
    print(f"Baseline: {baseline}")
    print(f"Steered: {steered}")
    
    results = compare_texts(baseline, steered)
    
    keyword_diff = results['steered_score'] - results['baseline_score']
    similarity_diff = results['steered_similarity'] - results['baseline_similarity']
    
    print(f"Keyword Scores - Baseline: {results['baseline_score']}, Steered: {results['steered_score']} (Diff: {keyword_diff})")
    print(f"Semantic Similarities - Baseline: {results['baseline_similarity']:.4f}, Steered: {results['steered_similarity']:.4f} (Diff: {similarity_diff:.4f})")
    print(evaluate_alignment(results))


Test Case 1:
Baseline: I think the most challenging career path is lawyer.
Steered: I think the most challenging career path is doctor.
Keyword Scores - Baseline: 0, Steered: 1 (Diff: 1)
Semantic Similarities - Baseline: 0.3739, Steered: 0.4201 (Diff: 0.0463)
The steered output is more aligned with the medical feature.

Test Case 2:
Baseline: I think the most challenging career path is lawyer.
Steered: I think the most challenging career path is nurse.
Keyword Scores - Baseline: 0, Steered: 1 (Diff: 1)
Semantic Similarities - Baseline: 0.3739, Steered: 0.4009 (Diff: 0.0270)
The steered output is more aligned with the medical feature.

Test Case 3:
Baseline: The project requires careful planning and execution.
Steered: The surgery requires careful planning and execution.
Keyword Scores - Baseline: 0, Steered: 1 (Diff: 1)
Semantic Similarities - Baseline: 0.3757, Steered: 0.4575 (Diff: 0.0817)
The steered output is more aligned with the medical feature.

Test Case 4:
Baseline: She works

In [19]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf

# Load the pre-trained model and tokenizer
model_name = "Humberto/MedicalArticlesClassificationModel"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

def classify_text(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="tf", truncation=True, padding=True, max_length=512)
    
    # Make a prediction
    outputs = model(inputs)
    
    # Get the predicted class (0 for non-medical, 1 for medical)
    predicted_class = tf.argmax(outputs.logits, axis=1).numpy()[0]
    
    # Get the confidence score
    confidence = tf.nn.softmax(outputs.logits, axis=1).numpy()[0][predicted_class]
    
    return predicted_class, confidence

# Test the model with some example texts
test_texts = [
    "The patient presented with symptoms of fever and cough.",
    "The stock market showed significant gains today.",
    "A new study reveals potential treatments for Alzheimer's disease.",
    "The movie received critical acclaim at the film festival."
]

for text in test_texts:
    class_label, confidence = classify_text(text)
    result = "Medical" if class_label == 1 else "Non-medical"
    print(f"Text: {text}")
    print(f"Classification: {result}")
    print(f"Confidence: {confidence:.2f}")
    print()

Some layers from the model checkpoint at Humberto/MedicalArticlesClassificationModel were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at Humberto/MedicalArticlesClassificationModel and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Text: The patient presented with symptoms of fever and cough.
Classification: Non-medical
Confidence: 0.18

Text: The stock market showed significant gains today.
Classification: Medical
Confidence: 0.18

Text: A new study reveals potential treatments for Alzheimer's disease.
Classification: Non-medical
Confidence: 0.18

Text: The movie received critical acclaim at the film festival.
Classification: Medical
Confidence: 0.18



In [21]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the pre-trained model and tokenizer
model_name = "justpyschitry/Medical_Article_Classifier_by_ICD-11_Chapter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)  # Remove from_pt=True

def classify_text(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    
    # Make a prediction
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the predicted class
    predicted_class = torch.argmax(outputs.logits, dim=1).item()
    
    # Get the confidence score
    confidence = torch.softmax(outputs.logits, dim=1)[0][predicted_class].item()
    
    return predicted_class, confidence

# Test the model with some example texts
test_texts = [
    "The patient presented with symptoms of fever and cough.",
    "The stock market showed significant gains today.",
    "A new study reveals potential treatments for Alzheimer's disease.",
    "The movie received critical acclaim at the film festival."
]

# Get the label mappings
id2label = model.config.id2label

for text in test_texts:
    class_label, confidence = classify_text(text)
    result = id2label[class_label]
    print(f"Text: {text}")
    print(f"Classification: {result}")
    print(f"Confidence: {confidence:.2f}")
    print()

Text: The patient presented with symptoms of fever and cough.
Classification: Certain infectious or parasitic diseases
Confidence: 0.95

Text: The stock market showed significant gains today.
Classification: Mental behavioural or neurodevelopmental disorders
Confidence: 0.19

Text: A new study reveals potential treatments for Alzheimer's disease.
Classification: diseases of the nervous system
Confidence: 0.72

Text: The movie received critical acclaim at the film festival.
Classification: Neoplasms
Confidence: 0.23

